### Load Historical SPX Options Prices via .csv files provided by Hanweck, Clean & Process Data  

In [2]:
import os
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

### Daily Data

In [46]:
filepath = os.getcwd() + r'\Data'
filepath

'C:\\Users\\David\\Documents\\IPython_Notebooks\\Capstone Project\\Data'

In [3]:
expiration = pd.read_csv(os.getcwd() + (r'\Data' + '\expiration_dates.csv'), parse_dates=['vx1', 'spx'], \
                         date_parser = lambda x: datetime.strptime(x,'%d-%b-%y'))

expiration.head()

,vx1,spx,Unnamed: 2
0,2005-10-19,2005-10-21,NaN
1,2005-11-16,2005-11-18,28.0
2,2005-12-21,2005-12-16,28.0
3,2006-01-18,2006-01-20,35.0
4,2006-02-15,2006-02-17,28.0


In [4]:
def get_expiration(month, year):
    for i in range(expiration.shape[0]):
        if expiration.iloc[i,0].month == month and expiration.iloc[i,0].year == year:
            return expiration.iloc[i,1]    

In [5]:
dateparser = lambda x: datetime.strptime(x, '%m/%d/%Y')

In [50]:
spxfiles = np.array(os.listdir(filepath + r'\SPX_raw'))
spxfiles

array(['SPX_2005-2010.csv', 'SPX_2011-2012.csv', 'SPX_2013.csv',
       'SPX_2014.csv', 'SPX_2015_01-06.csv', 'SPX_2015_07-12.csv',
       'SPX_2016_01-04.csv'], 
      dtype='<U18')

In [51]:
spx = pd.DataFrame()
for file in spxfiles:
    print(file)
    filename = os.path.join(filepath, 'SPX_raw', file)
    #df = pd.read_csv(filename, usecols=[0,2,3,4,9,10], parse_dates=['date'], date_parser = dateparser)
    df = pd.read_csv(filename, usecols=[0,2,3,4,9,10])
    df = df[(np.abs(df['delta']) > 0.4) & (np.abs(df['delta']) < 0.6)]
    df['date'] = df['date'].apply(dateparser)
    df['exdate'] = df['exdate'].apply(dateparser)
    df['exdate'] = df['exdate'].apply(lambda x: x - pd.Timedelta(days=1) if x.weekday_name == 'Saturday' else x)
    df = df[(df['exdate'].isin(expiration['spx']))]
    #print(expiry_list)

    spx = pd.concat([spx,df])

SPX_2005-2010.csv
SPX_2011-2012.csv
SPX_2013.csv
SPX_2014.csv
SPX_2015_01-06.csv
SPX_2015_07-12.csv
SPX_2016_01-04.csv


In [52]:
x = spx.iloc[0,1]
print(x)

x - pd.Timedelta(days=1)

2005-10-21 00:00:00


Timestamp('2005-10-20 00:00:00')

In [53]:
spx.shape

(144759, 6)

In [54]:
spx.sort_values(['date'])
spx[100:120]

#type(spx['date'])
spx.dtypes

date               datetime64[ns]
exdate             datetime64[ns]
cp_flag                    object
strike_price                int64
impl_volatility           float64
delta                     float64
dtype: object

In [55]:
#spx['year_month'] = spx['date'].map(lambda x: [x.year, x.month])    
#spx['pre_expiry'] = spx['date'].map(lambda x: x <= expiration['vx1'].loc((expiration['vx1'].dt.year == spx['date'].dt.year) & \
                                   #(expiration['vx1'].dt.month == spx['date'].dt.month))) 
spx.head()

,date,exdate,cp_flag,strike_price,impl_volatility,delta
10550,2005-07-18,2005-10-21,C,1225,0.117756,0.523127
10551,2005-07-18,2005-10-21,P,1225,0.109906,-0.472197
10808,2005-07-19,2005-10-21,C,1225,0.113671,0.568894
10809,2005-07-19,2005-10-21,P,1225,0.111125,-0.425542
10810,2005-07-19,2005-10-21,C,1250,0.105629,0.423009


In [56]:
#spx['pre_expiry'] = False*spx.shape[0]

if 0:
    for i in expiration['vx1']:
        yr = i.year
        mnth = i.month
        d = i.day
        slic = ((spx['date'].dt.year == yr) & (spx['date'].dt.month == mnth))
        spx['pre_expiry'][slic] = spx['date'].map(lambda x: x.day <= d)    

In [57]:
#spx[500:600]
#spx[spx['pre_expiry'] == 1] 

In [58]:
spx.tail(10)

,date,exdate,cp_flag,strike_price,impl_volatility,delta
470350,2016-04-29,2016-08-19,C,2050,0.154533,0.526637
470351,2016-04-29,2016-08-19,P,2050,0.159986,-0.466648
470352,2016-04-29,2016-08-19,C,2060,0.151627,0.503533
470353,2016-04-29,2016-08-19,P,2060,0.156886,-0.489000
470354,2016-04-29,2016-08-19,C,2075,0.147030,0.467290
470355,2016-04-29,2016-08-19,P,2075,0.152145,-0.524061
470356,2016-04-29,2016-08-19,C,2080,0.145377,0.454746
470357,2016-04-29,2016-08-19,P,2080,0.150750,-0.536051
470358,2016-04-29,2016-08-19,C,2100,0.138985,0.402720
470359,2016-04-29,2016-08-19,P,2100,0.144681,-0.585892


#### Designate a time slice/start date to construct a subset of options data for procedure testing 

In [59]:
# Designated time slice
if 0:
    spxTest = spx[(spx['date'] > pd.to_datetime('20051231', format='%Y%m%d', errors='ignore'))& \
                  (spx['date'] < pd.to_datetime('20070101', format='%Y%m%d', errors='ignore'))]
    spxTest.shape

In [60]:
# Designated start date (other than beginning of historical data)
if 1:
    spx = spx[spx['date'] > pd.to_datetime('20050930', format='%Y%m%d', errors='ignore')]

In [13]:
# Define column numbers for VIX, SPX expiration dates in 'expiration_dates.csv'
vx_expirys = 0
sp_expirys = 1

def findTerms(date):
    for i in range(expiration.shape[vx_expirys]): # Determine month & year of VIX expiry corresponding to current date
        if expiration.iloc[i,vx_expirys].month == date.month and expiration.iloc[i,vx_expirys].year == date.year:
            break                                 # Effectively reduces search area of posible dates
    
    # Determine EXCEPTION CASES: 4 Times/yr 1st SPX option expiry falls BEFORE VIX expiry
    if expiration.iloc[i,sp_expirys] < expiration.iloc[i,vx_expirys]: 
        if date >= expiration.iloc[i,vx_expirys]:
            vix_expiry = expiration.iloc[i+1,vx_expirys]
            term1 = expiration.iloc[i+1,sp_expirys]
            term2 = expiration.iloc[i+2,sp_expirys]
        else:
            vix_expiry = expiration.iloc[i+0,vx_expirys]
            term1 = expiration.iloc[i+0,sp_expirys] + pd.Timedelta(days=7)
            term2 = expiration.iloc[i+1,sp_expirys]
    elif expiration.iloc[i+1,sp_expirys] < expiration.iloc[i+1,vx_expirys]:
        if date < expiration.iloc[i,vx_expirys]:
            vix_expiry = expiration.iloc[i+0,vx_expirys]
            term1 = expiration.iloc[i+0,sp_expirys]
            term2 = expiration.iloc[i+1,sp_expirys]
        else:
            vix_expiry = expiration.iloc[i+1,vx_expirys]
            term1 = expiration.iloc[i+1,sp_expirys] + pd.Timedelta(days=7)
            term2 = expiration.iloc[i+2,sp_expirys]
    else:
        if date >= expiration.iloc[i,vx_expirys]:
            vix_expiry = expiration.iloc[i+1,vx_expirys]
            term1 = expiration.iloc[i+1,sp_expirys]
            term2 = expiration.iloc[i+2,sp_expirys]
        else:
            vix_expiry = expiration.iloc[i+0,vx_expirys]
            term1 = expiration.iloc[i+0,sp_expirys]
            term2 = expiration.iloc[i+1,sp_expirys]


    return pd.Series([vix_expiry, term1, term2], index=['VIX_Expiry', 'Term_1', 'Term_2'])

In [66]:
# vix      ,  spx
#19-Aug-15	21-Aug-15
#16-Sep-15	18-Sep-15
# 21-Oct-15	16-Oct-15
#18-Nov-15	20-Nov-15

date = datetime(2005,10,3)
findTerms(date)

VIX_Expiry   2005-10-19
Term_1       2005-10-21
Term_2       2005-11-18
dtype: datetime64[ns]

In [67]:
# Apply function to data subset
#terms = spxTest['date'].apply(findTerms)

# Apply function to full options dataset
terms = spx['date'].apply(findTerms)

#### Combine options data with 2 designated option expiration dates (Term_1, Term_2) surrounding VIX futures contract expiration

In [69]:
# Shorten orginal spx dataframe to include [all rows(':'), columns 0-5(':6')]
spx = spx.iloc[:,:6]

In [70]:
# Data subset
#spxTest = pd.concat([spxTest, terms], axis=1)

# FULL dataset
spx = pd.concat([spx, terms], axis=1)
spx.head()

,date,exdate,cp_flag,strike_price,impl_volatility,delta,VIX_Expiry,Term_1,Term_2
27920,2005-10-03,2005-10-21,C,1225,0.113678,0.541939,2005-10-19,2005-10-21,2005-11-18
27921,2005-10-03,2005-10-21,P,1225,0.109207,-0.455839,2005-10-19,2005-10-21,2005-11-18
27922,2005-10-03,2005-10-21,C,1230,0.109342,0.474535,2005-10-19,2005-10-21,2005-11-18
27923,2005-10-03,2005-10-21,P,1230,0.103549,-0.526490,2005-10-19,2005-10-21,2005-11-18
27924,2005-10-03,2005-10-21,C,1235,0.106459,0.404139,2005-10-19,2005-10-21,2005-11-18


#### OPTIONAL - Dataframe Indexing

In [22]:
#spxTest = spxTest.set_index('date')
#spx = spx.set_index('date')

# RESET Data subset index
#spxTest = spxTest.reset_index()

# RESET FULL dataset index
#spx = spx.reset_index()

In [71]:
#spxTest[236:276]

spx.head()

,date,exdate,cp_flag,strike_price,impl_volatility,delta,VIX_Expiry,Term_1,Term_2
27920,2005-10-03,2005-10-21,C,1225,0.113678,0.541939,2005-10-19,2005-10-21,2005-11-18
27921,2005-10-03,2005-10-21,P,1225,0.109207,-0.455839,2005-10-19,2005-10-21,2005-11-18
27922,2005-10-03,2005-10-21,C,1230,0.109342,0.474535,2005-10-19,2005-10-21,2005-11-18
27923,2005-10-03,2005-10-21,P,1230,0.103549,-0.526490,2005-10-19,2005-10-21,2005-11-18
27924,2005-10-03,2005-10-21,C,1235,0.106459,0.404139,2005-10-19,2005-10-21,2005-11-18


#### Find ATM implied vols (Option1, Option2) & trading days to expiration (t1, t2) for Term 1 & Term 2 options

In [72]:
df

,date,exdate,cp_flag,strike_price,impl_volatility,delta
725,2016-01-04,2016-01-15,P,1995,0.225079,-0.403842
726,2016-01-04,2016-01-15,C,2000,0.204103,0.575031
727,2016-01-04,2016-01-15,P,2000,0.222720,-0.429470
728,2016-01-04,2016-01-15,C,2005,0.200594,0.546485
729,2016-01-04,2016-01-15,P,2005,0.219403,-0.455758
730,2016-01-04,2016-01-15,C,2010,0.197904,0.516730
731,2016-01-04,2016-01-15,P,2010,0.216236,-0.482985
732,2016-01-04,2016-01-15,C,2015,0.195975,0.486239
733,2016-01-04,2016-01-15,P,2015,0.214303,-0.510882
734,2016-01-04,2016-01-15,C,2020,0.190988,0.454392


In [62]:
if 0:
    term1 = df[df['exdate'] == df['Term_1'].iloc[0]]
    term2 = df[df['exdate'] == df['Term_2'].iloc[0]]
    t1 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['Term_1'].iloc[0] )])
    t2 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['Term_2'].iloc[0] )])

In [137]:
term2 = df[df['exdate'] == df['Term_2'].iloc[0]]
term2.head()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price,VIX_Expiry,Term_1,Term_2


In [32]:
def findOptions(df):
    term1 = df[df['exdate'] == df['Term_1'].iloc[0]]
    term2 = df[df['exdate'] == df['Term_2'].iloc[0]]
    #t1 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['VIX_Expiry'].iloc[0])]) 
    #t2 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['Term_2'].iloc[0] )])
    
    # Calculate current timestamp & future VIX expirys out to the hour & minute 
    # This approach allows for a more continuous view of Time to Expiry (TTE)
    partial_day = (df['date'].iloc[0].hour + df['date'].iloc[0].minute/60)/24 - (9.5)/24
    t1 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['VIX_Expiry'].iloc[0])]) - partial_day
    t2 = len(trade_dates[(trade_dates > df['date'].iloc[0]) & ( trade_dates <= df['Term_2'].iloc[0] )]) - partial_day
    
    if term1.shape[0] == 0 or term2.shape[0] == 0:
        return pd.Series([np.nan, np.nan, np.nan, np.nan, t1, t2], index=['Strike1', 'Strike2','Option1', 'Option2', 't1', 't2'])
        
        
    #print(df['date'].iloc[0])
    strike_list1 = np.unique(term1['strike_price'])
    #print(strike_list1)
    mindelta1 = 5
    for strikeIndx, strike in enumerate(strike_list1): 
        x = np.abs(term1[term1['strike_price'] == strike]['delta'].sum())
        if x < mindelta1:
            mindelta1 = x
            minStrike1 = strike
            
    strike_list2 = np.unique(term2['strike_price'])
    mindelta2 = 5
    for strikeIndx, strike in enumerate(strike_list2): 
        x = np.abs(term2[term2['strike_price'] == strike]['delta'].sum())
        if x < mindelta2:
            mindelta2 = x
            minStrike2 = strike
    avg1 = term1[term1['strike_price'] == minStrike1]['impl_volatility'].mean()
    avg2 = term2[term2['strike_price'] == minStrike2]['impl_volatility'].mean()
    
    return pd.Series([minStrike1, minStrike2, avg1, avg2, t1, t2], index=['Strike1', 'Strike2','Option1', 'Option2', 't1', 't2'])  

In [36]:
intra.iloc[:5040,:].groupby('date').apply(findOptions)

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2015-01-02 09:40:00,2070.0,2070.0,0.132620,0.136980,11.993056,32.993056
2015-01-02 09:50:00,2070.0,2070.0,0.132455,0.136215,11.986111,32.986111
2015-01-02 10:00:00,2070.0,2065.0,0.134345,0.138615,11.979167,32.979167
2015-01-02 10:10:00,2065.0,2060.0,0.139675,0.143980,11.972222,32.972222
2015-01-02 10:20:00,2065.0,2060.0,0.140710,0.144930,11.965278,32.965278
2015-01-02 10:30:00,2065.0,2065.0,0.141665,0.143260,11.958333,32.958333
2015-01-02 10:40:00,2055.0,2055.0,0.152860,0.150710,11.951389,32.951389
2015-01-02 10:50:00,2055.0,2055.0,0.156325,0.153460,11.944444,32.944444
2015-01-02 11:00:00,2055.0,2055.0,0.151820,0.151885,11.937500,32.937500


0.40277777777777773

#### Calculate trade dates

In [74]:
# trade dates for data subset
#trade_dates = pd.DataFrame(np.unique(spxTest['date']))

# trade dates for FULL dataset
trade_dates = pd.DataFrame(np.unique(spx['date']))

trade_dates.head()

,0
0,2005-10-03
1,2005-10-04
2,2005-10-05
3,2005-10-06
4,2005-10-07


In [75]:
trade_dates = np.array([pd.Timestamp(x) for x in np.unique(spx['date'])])
#trade_dates = np.array([pd.Timestamp(x) for x in np.unique(spx.index)])
len(trade_dates)

2662

In [33]:
#trade_dates[trade_dates > spxTest.iloc[-1,0]]

In [34]:
#df = spxTest.groupby('date').get_group('2006-12-28')
#type(findOptions(df))

In [35]:
#spx_tdate = spx[spx['date'] == '2016-03-18']
#spx_tdate.head()

#### Apply 'findOptions' function to dataset

In [76]:
spx[spx['date'] == '2015-10-19']['strike_price'].unique

<bound method Series.unique of 520809    2015
520810    2015
520811    2020
520812    2020
520813    2025
520814    2025
520815    2030
520816    2030
520817    2035
520818    2035
520819    2040
520820    2040
520821    2045
520822    2045
520824    2050
522837    2005
522838    2005
522839    2010
522840    2010
522841    2015
522842    2015
522843    2020
522844    2020
522845    2025
522846    2025
522847    2030
522848    2030
522849    2035
522850    2035
522851    2040
          ... 
522857    2055
522858    2055
524413    1995
524414    1995
524415    2000
524416    2000
524417    2005
524418    2005
524419    2010
524420    2010
524421    2015
524422    2015
524423    2020
524424    2020
524425    2025
524426    2025
524427    2030
524428    2030
524429    2035
524430    2035
524431    2040
524432    2040
524433    2045
524434    2045
524435    2050
524436    2050
524437    2055
524438    2055
524439    2060
524440    2060
Name: strike_price, Length: 65, dtype: int64>

In [77]:
# Apply function to data subset
if 0:
    options = spxTest.groupby('date').apply(findOptions)

# Apply function to FULL options dataset
if 1:
    options = spx.groupby('date').apply(findOptions)

In [24]:
#spx.head()

In [78]:
options.head()

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2005-10-03,1230.0,1230.0,0.106445,0.113757,12.0,34.0
2005-10-04,1215.0,1220.0,0.119392,0.120574,11.0,33.0
2005-10-05,1200.0,1200.0,0.136211,0.133042,10.0,32.0
2005-10-06,1195.0,1200.0,0.143999,0.133893,9.0,31.0
2005-10-07,1195.0,1200.0,0.140486,0.132000,8.0,30.0


In [39]:
#options.loc[:,['Strike1', 'Strike2','Option1','Option2']] = options.loc[:,['Strike1', 'Strike2','Option1','Option2']].interpolate()
#options.head(10)
#options.tail(50)

#### Combine 'options' & select columns from 'spx' in new dataframe 'spx1' and export via pickle 

In [79]:
# Add 'Term1' & 'Term2' columns from 'spx' to 'options' (they should have the same index) to create new 'spx1' dataframe
#spx1 = 

spx_opt = options

In [80]:
spx_opt.head()

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2005-10-03,1230.0,1230.0,0.106445,0.113757,12.0,34.0
2005-10-04,1215.0,1220.0,0.119392,0.120574,11.0,33.0
2005-10-05,1200.0,1200.0,0.136211,0.133042,10.0,32.0
2005-10-06,1195.0,1200.0,0.143999,0.133893,9.0,31.0
2005-10-07,1195.0,1200.0,0.140486,0.132000,8.0,30.0


In [82]:
spx1 = spx_opt[:'2016-03-09']

In [83]:
spx1.tail()

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2016-03-03,1995.0,1990.0,0.155378,0.155438,9.0,30.0
2016-03-04,2000.0,2000.0,0.156439,0.153197,8.0,29.0
2016-03-07,2000.0,2000.0,0.169019,0.156206,7.0,28.0
2016-03-08,1980.0,1980.0,0.186869,0.168959,6.0,27.0
2016-03-09,1990.0,1990.0,0.185499,0.165777,5.0,26.0


#### Save Daily data to pickle

In [84]:
#spx1.to_pickle(r'C:\Users\David\Documents\IPython_Notebooks\Capstone Project\Data\spx1.pkl')
spx1.to_pickle(filepath + r'\spx1.pkl')

### Intraday Data

In [7]:
intra_filepath = os.getcwd() + r'\Data\Intraday'
intra_filepath

'C:\\Users\\David\\Documents\\IPython_Notebooks\\Capstone Project\\Data\\Intraday'

In [98]:
spx_intra_files = np.array(os.listdir(intra_filepath + r'\SPX_raw'))
spx_intra_files

array(['SPX with vols greeks hamilton 2015.csv',
       'SPX with vols greeks hamilton 201601 - 201603.csv',
       'SPX with vols greeks hamilton 201604 - 201612.csv'], 
      dtype='<U49')

In [99]:
intra = pd.DataFrame()
for file in spx_intra_files:
    print(file)
    filename = os.path.join(intra_filepath, 'SPX_raw', file)
    #df = pd.read_csv(filename, usecols=[0,2,3,4,9,10], parse_dates=['date'], date_parser = dateparser)
    df = pd.read_csv(filename, parse_dates=['TIMESTAMP'],\
                    date_parser = lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'),\
                    usecols = ['SYMBOL', 'TIMESTAMP', 'MID_VOL', 'DELTA'])
    df = df[(np.abs(df['DELTA']) > 0.38) & (np.abs(df['DELTA']) < 0.62)]
    #df['date'] = df['date'].apply(dateparser)
    #df['exdate'] = df['exdate'].apply(dateparser)
    #df['exdate'] = df['exdate'].apply(lambda x: x - pd.Timedelta(days=1) if x.weekday_name == 'Saturday' else x)
    #df = df[(df['exdate'].isin(expiration['spx']))]
    #print(expiry_list)

    intra = pd.concat([intra,df])

SPX with vols greeks hamilton 2015.csv
SPX with vols greeks hamilton 201601 - 201603.csv
SPX with vols greeks hamilton 201604 - 201612.csv


In [100]:
if 0:
    intra = pd.read_csv(intra_filepath + '\SPX_with_vols_greeks_hamilton.csv', parse_dates=['TIMESTAMP'],\
                    date_parser = lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'),\
                    usecols = ['SYMBOL', 'TIMESTAMP', 'MID_VOL', 'DELTA'])

In [101]:
intra.rename(columns={'TIMESTAMP': 'date', 'DELTA':'delta', 'MID_VOL':'impl_volatility'}, inplace=True)

In [24]:
#intra_monthly = pd.read_csv(intra_filepath + '\SPX options vols and greeks.csv', parse_dates=['TIMESTAMP'], \
#                     date_parser = lambda x: datetime.strptime(x,'%m/%d/%Y %H:%M'), 
#                     usecols = ['SYMBOL', 'TIMESTAMP', 'MID_VOL', 'DELTA'])

#intra_weekly = pd.read_csv(intra_filepath + '\SPXW options price vols and greeks.csv', parse_dates=['TIMESTAMP'], \
#                     date_parser = lambda x: datetime.strptime(x,'%m/%d/%Y %H:%M'), 
#                     usecols = ['SYMBOL', 'TIMESTAMP', 'MID_VOL', 'DELTA'])


#intra = pd.concat([intra_monthly, intra_weekly])
#intra.rename(columns={'TIMESTAMP': 'date', 'DELTA':'delta', 'MID_VOL':'impl_volatility'}, inplace=True)

In [102]:
intra.dtypes

SYMBOL                     object
date               datetime64[ns]
impl_volatility           float64
delta                     float64
dtype: object

#### Screen data by instituting hi/lo delta thresholds

In [35]:
#lo_delta = 0.35
#hi_delta = 0.65

In [36]:
#intra = intra[(intra['delta'].abs() >= lo_delta) & (intra['delta'].abs() <= hi_delta) ]

#### Define function to split on space following 'SPX/SPXW..." and take 2nd part of OPRA code: YY-MM-DD, Call/Put, Strike Price 

In [10]:
def parse_symbol(x):
    x = x.split()[1]
    date = datetime.strptime(x[:6],'%y%m%d')
    if date.weekday() == 5:
        date -= pd.Timedelta(days=1)
    optiontype = x[6]
    strike = float(x[8:12])
    return pd.Series([date, optiontype,strike], index=['exdate', 'cp_flag', 'strike_price'])    

#### Parse expiration date, Call/Put flag & strike price from OPRA symbol

In [104]:
intra[['exdate', 'cp_flag', 'strike_price']] = intra['SYMBOL'].apply(parse_symbol)
intra.head(10)

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price
4424,SPXW 150123C02025000,2015-01-02 13:30:00,0.16676,0.61442,2015-01-23,C,2025.0
4425,SPXW 150123C02025000,2015-01-02 13:40:00,0.16949,0.60985,2015-01-23,C,2025.0
4426,SPXW 150123C02025000,2015-01-02 13:50:00,0.16891,0.61068,2015-01-23,C,2025.0
4464,SPXW 150123P02025000,2015-01-02 13:30:00,0.16765,-0.38631,2015-01-23,P,2025.0
4465,SPXW 150123P02025000,2015-01-02 13:40:00,0.16830,-0.38943,2015-01-23,P,2025.0
4466,SPXW 150123P02025000,2015-01-02 13:50:00,0.16816,-0.38911,2015-01-23,P,2025.0
4490,SPXW 150123C02030000,2015-01-02 11:10:00,0.17330,0.60974,2015-01-23,C,2030.0
4491,SPXW 150123C02030000,2015-01-02 11:20:00,0.17174,0.60059,2015-01-23,C,2030.0
4492,SPXW 150123C02030000,2015-01-02 11:30:00,0.16957,0.61265,2015-01-23,C,2030.0
4493,SPXW 150123C02030000,2015-01-02 11:40:00,0.16627,0.61323,2015-01-23,C,2030.0


In [105]:
intra.tail()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price
10931816,SPX 170217P02270000,2016-12-30 12:10:00,0.11231,-0.61276,2017-02-17,P,2270.0
10931817,SPX 170217P02270000,2016-12-30 12:20:00,0.11137,-0.61345,2017-02-17,P,2270.0
10931818,SPX 170217P02270000,2016-12-30 12:30:00,0.11087,-0.61582,2017-02-17,P,2270.0
10931819,SPX 170217P02270000,2016-12-30 12:40:00,0.11209,-0.61831,2017-02-17,P,2270.0
10931820,SPX 170217P02270000,2016-12-30 12:50:00,0.11274,-0.61890,2017-02-17,P,2270.0


In [106]:
intra.sort_values(by =['date', 'exdate', 'strike_price', 'cp_flag'], inplace = True)
intra.head()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price
4801,SPXW 150123C02050000,2015-01-02 09:40:00,0.14808,0.61873,2015-01-23,C,2050.0
4881,SPXW 150123C02055000,2015-01-02 09:40:00,0.14306,0.59702,2015-01-23,C,2055.0
4921,SPXW 150123P02055000,2015-01-02 09:40:00,0.14244,-0.40365,2015-01-23,P,2055.0
4961,SPXW 150123C02060000,2015-01-02 09:40:00,0.13922,0.57183,2015-01-23,C,2060.0
5001,SPXW 150123P02060000,2015-01-02 09:40:00,0.14016,-0.42949,2015-01-23,P,2060.0


In [107]:
#intra = intra.iloc[:10000,:]
intra.shape

(2380348, 7)

In [108]:
intra.tail()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price
10931599,SPX 170217P02255000,2016-12-30 16:00:00,0.11198,-0.57027,2017-02-17,P,2255.0
10931639,SPX 170217C02260000,2016-12-30 16:00:00,0.11629,0.41325,2017-02-17,C,2260.0
10931679,SPX 170217P02260000,2016-12-30 16:00:00,0.11014,-0.59306,2017-02-17,P,2260.0
10931719,SPX 170217C02265000,2016-12-30 16:00:00,0.11559,0.39250,2017-02-17,C,2265.0
10931759,SPX 170217P02265000,2016-12-30 16:00:00,0.11000,-0.61424,2017-02-17,P,2265.0


In [109]:
#intra.to_pickle(intra_filepath + r'\Pickle\spx_intraday(mid).pkl')

#### Find Correct Options Term 1 & Term 2

In [87]:
# TEST CASE
#intra = intra.iloc[:10000,:7]

In [110]:
intra[['VIX_Expiry', 'Term_1', 'Term_2']]= intra['date'].apply(findTerms)

In [111]:
intra.head()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price,VIX_Expiry,Term_1,Term_2
4801,SPXW 150123C02050000,2015-01-02 09:40:00,0.14808,0.61873,2015-01-23,C,2050.0,2015-01-21,2015-01-23,2015-02-20
4881,SPXW 150123C02055000,2015-01-02 09:40:00,0.14306,0.59702,2015-01-23,C,2055.0,2015-01-21,2015-01-23,2015-02-20
4921,SPXW 150123P02055000,2015-01-02 09:40:00,0.14244,-0.40365,2015-01-23,P,2055.0,2015-01-21,2015-01-23,2015-02-20
4961,SPXW 150123C02060000,2015-01-02 09:40:00,0.13922,0.57183,2015-01-23,C,2060.0,2015-01-21,2015-01-23,2015-02-20
5001,SPXW 150123P02060000,2015-01-02 09:40:00,0.14016,-0.42949,2015-01-23,P,2060.0,2015-01-21,2015-01-23,2015-02-20


In [112]:
intra.tail()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price,VIX_Expiry,Term_1,Term_2
10931599,SPX 170217P02255000,2016-12-30 16:00:00,0.11198,-0.57027,2017-02-17,P,2255.0,2017-01-18,2017-01-20,2017-02-17
10931639,SPX 170217C02260000,2016-12-30 16:00:00,0.11629,0.41325,2017-02-17,C,2260.0,2017-01-18,2017-01-20,2017-02-17
10931679,SPX 170217P02260000,2016-12-30 16:00:00,0.11014,-0.59306,2017-02-17,P,2260.0,2017-01-18,2017-01-20,2017-02-17
10931719,SPX 170217C02265000,2016-12-30 16:00:00,0.11559,0.39250,2017-02-17,C,2265.0,2017-01-18,2017-01-20,2017-02-17
10931759,SPX 170217P02265000,2016-12-30 16:00:00,0.11000,-0.61424,2017-02-17,P,2265.0,2017-01-18,2017-01-20,2017-02-17


#### Pickle initial 'intra' dataframe

In [113]:
#intra.to_pickle(intra_filepath + r'\Pickle\spx_intraday(mid).pkl')

#### Load initial 'intra' dataframe from pickle

In [8]:
intra = pickle.load(open(intra_filepath + r'\Pickle\spx_intraday(mid).pkl', "rb" ))

#### Insert missing time series data for 2015-12-18 expiration from 10/21/2015 through 11/17/2015

In [11]:
# Read file into Pandas dataframe
filename = intra_filepath + r'\SPX with vols greeks hamilton(Dec-15).csv'
df = pd.read_csv(filename, parse_dates=['TIMESTAMP'],\
                    date_parser = lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'),\
                    usecols = ['SYMBOL', 'TIMESTAMP', 'MID_VOL', 'DELTA'])

# Screen for options with absolute delta > 0.38 & < 0.62
df = df[(np.abs(df['DELTA']) > 0.38) & (np.abs(df['DELTA']) < 0.62)]

# Rename columns to match master 'intra' dataframe
df.rename(columns={'TIMESTAMP': 'date', 'DELTA':'delta', 'MID_VOL':'impl_volatility'}, inplace=True)

# Parse Expiration date, Call/Put designation & Stike Price from OPRA symbol
df[['exdate', 'cp_flag', 'strike_price']] = df['SYMBOL'].apply(parse_symbol)

# Sort values by trade date, expiration, strike & Call/Put
df.sort_values(by =['date', 'exdate', 'strike_price', 'cp_flag'], inplace = True)

In [14]:
df[['VIX_Expiry', 'Term_1', 'Term_2']]= df['date'].apply(findTerms)

In [15]:
intra = pd.concat([intra, df])

In [16]:
intra.sort_values(by =['date', 'exdate', 'strike_price', 'cp_flag'], inplace = True)

#### Parse expiration date, Call/Put flag & strike price from OPRA symbol for weekly options

In [59]:
#df = intra[intra['date'] == '2014-10-22 10:00:00']
#df.shape

(544, 9)

In [60]:
#intra['date'] = intra['TIMESTAMP'].apply(lambda x: datetime.strptime(x.strftime('%Y-%m-%d'),'%Y-%m-%d'))

#### Calculate trade dates for intraday dataset

In [17]:
trade_dates = np.array([pd.Timestamp(x) for x in np.unique(intra['date'].apply(lambda x: x.strftime('%Y-%m-%d')))])

In [18]:
trade_dates[-5:]

array([Timestamp('2016-12-23 00:00:00'), Timestamp('2016-12-27 00:00:00'),
       Timestamp('2016-12-28 00:00:00'), Timestamp('2016-12-29 00:00:00'),
       Timestamp('2016-12-30 00:00:00')], dtype=object)

#### Insert additional trade dates for additional 1.5 months after end of official dataset (to calculate time series)

In [19]:
extra_dates = pd.date_range('2017-01-01', '2017-02-17', freq=pd.tseries.offsets.BDay()).tolist()
extra_dates

[Timestamp('2017-01-02 00:00:00', freq='B'),
 Timestamp('2017-01-03 00:00:00', freq='B'),
 Timestamp('2017-01-04 00:00:00', freq='B'),
 Timestamp('2017-01-05 00:00:00', freq='B'),
 Timestamp('2017-01-06 00:00:00', freq='B'),
 Timestamp('2017-01-09 00:00:00', freq='B'),
 Timestamp('2017-01-10 00:00:00', freq='B'),
 Timestamp('2017-01-11 00:00:00', freq='B'),
 Timestamp('2017-01-12 00:00:00', freq='B'),
 Timestamp('2017-01-13 00:00:00', freq='B'),
 Timestamp('2017-01-16 00:00:00', freq='B'),
 Timestamp('2017-01-17 00:00:00', freq='B'),
 Timestamp('2017-01-18 00:00:00', freq='B'),
 Timestamp('2017-01-19 00:00:00', freq='B'),
 Timestamp('2017-01-20 00:00:00', freq='B'),
 Timestamp('2017-01-23 00:00:00', freq='B'),
 Timestamp('2017-01-24 00:00:00', freq='B'),
 Timestamp('2017-01-25 00:00:00', freq='B'),
 Timestamp('2017-01-26 00:00:00', freq='B'),
 Timestamp('2017-01-27 00:00:00', freq='B'),
 Timestamp('2017-01-30 00:00:00', freq='B'),
 Timestamp('2017-01-31 00:00:00', freq='B'),
 Timestamp

#### Remove MLK Holiday as a trade date

In [20]:
extra_dates.pop(10)

Timestamp('2017-01-16 00:00:00', freq='B')

In [21]:
#extra_dates
trade_dates = np.append(trade_dates, extra_dates)

In [22]:
trade_dates[-5:]

array([Timestamp('2017-02-13 00:00:00', freq='B'),
       Timestamp('2017-02-14 00:00:00', freq='B'),
       Timestamp('2017-02-15 00:00:00', freq='B'),
       Timestamp('2017-02-16 00:00:00', freq='B'),
       Timestamp('2017-02-17 00:00:00', freq='B')], dtype=object)

In [23]:
x = intra[intra['date'] == '2015-01-02 09:40:00']
x.shape

(105, 10)

In [32]:
findOptions(x)

Strike1    2070.00000
Strike2    2070.00000
Option1       0.13262
Option2       0.13698
t1           12.00000
t2           33.00000
dtype: float64

In [43]:
df_2 = intra[intra['date'] == '2015-10-21 09:40:00']
#df_2 = intra[intra['date'] == '2015-11-18 09:50:00']
df_2.tail()

,SYMBOL,date,impl_volatility,delta,exdate,cp_flag,strike_price,VIX_Expiry,Term_1,Term_2
5321,SPX 151219P02050000,2015-10-21 09:40:00,0.13310,-0.57230,2015-12-18,P,2050.0,2015-11-18,2015-11-20,2015-12-18
5361,SPX 151219C02055000,2015-10-21 09:40:00,0.13123,0.40796,2015-12-18,C,2055.0,2015-11-18,2015-11-20,2015-12-18
5401,SPX 151219P02055000,2015-10-21 09:40:00,0.13131,-0.59161,2015-12-18,P,2055.0,2015-11-18,2015-11-20,2015-12-18
5441,SPX 151219C02060000,2015-10-21 09:40:00,0.12926,0.38822,2015-12-18,C,2060.0,2015-11-18,2015-11-20,2015-12-18
5481,SPX 151219P02060000,2015-10-21 09:40:00,0.12919,-0.61151,2015-12-18,P,2060.0,2015-11-18,2015-11-20,2015-12-18


In [34]:
df_2['exdate'].unique()

array(['2015-11-20T00:00:00.000000000', '2015-11-27T00:00:00.000000000',
       '2015-11-30T00:00:00.000000000', '2015-12-04T00:00:00.000000000',
       '2015-12-11T00:00:00.000000000', '2015-12-18T00:00:00.000000000'], dtype='datetime64[ns]')

In [35]:
findOptions(df_2)

Strike1    2030.000000
Strike2    2030.000000
Option1       0.130415
Option2       0.140770
t1           20.000000
t2           41.000000
dtype: float64

In [37]:
options = intra.groupby('date').apply(findOptions)

In [38]:
options.head(10)

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2015-01-02 09:40:00,2070.0,2070.0,0.132620,0.136980,11.993056,32.993056
2015-01-02 09:50:00,2070.0,2070.0,0.132455,0.136215,11.986111,32.986111
2015-01-02 10:00:00,2070.0,2065.0,0.134345,0.138615,11.979167,32.979167
2015-01-02 10:10:00,2065.0,2060.0,0.139675,0.143980,11.972222,32.972222
2015-01-02 10:20:00,2065.0,2060.0,0.140710,0.144930,11.965278,32.965278
2015-01-02 10:30:00,2065.0,2065.0,0.141665,0.143260,11.958333,32.958333
2015-01-02 10:40:00,2055.0,2055.0,0.152860,0.150710,11.951389,32.951389
2015-01-02 10:50:00,2055.0,2055.0,0.156325,0.153460,11.944444,32.944444
2015-01-02 11:00:00,2055.0,2055.0,0.151820,0.151885,11.937500,32.937500


In [39]:
options.tail(10)

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2016-12-30 14:30:00,2240.0,2235.0,0.115805,0.125360,11.791667,33.791667
2016-12-30 14:40:00,2240.0,2240.0,0.115150,0.123240,11.784722,33.784722
2016-12-30 14:50:00,2235.0,2235.0,0.116870,0.124695,11.777778,33.777778
2016-12-30 15:00:00,2235.0,2235.0,0.117395,0.124895,11.770833,33.770833
2016-12-30 15:10:00,2235.0,2235.0,0.118525,0.124730,11.763889,33.763889
2016-12-30 15:20:00,2240.0,2235.0,0.114790,0.124350,11.756944,33.756944
2016-12-30 15:30:00,2240.0,2240.0,0.112995,0.121940,11.750000,33.750000
2016-12-30 15:40:00,2240.0,2235.0,0.112185,0.123280,11.743056,33.743056
2016-12-30 15:50:00,2235.0,2235.0,0.114750,0.123050,11.736111,33.736111


In [40]:
options['2015-10-20':'2015-10-21']

,Strike1,Strike2,Option1,Option2,t1,t2
date,,,,,,
2015-10-20 09:40:00,2025.0,2025.0,0.135095,0.131890,0.993056,22.993056
2015-10-20 09:50:00,2030.0,2030.0,0.126825,0.128715,0.986111,22.986111
2015-10-20 10:00:00,2035.0,2030.0,0.121300,0.129155,0.979167,22.979167
2015-10-20 10:10:00,2035.0,2030.0,0.118680,0.128755,0.972222,22.972222
2015-10-20 10:20:00,2035.0,2030.0,0.118555,0.128555,0.965278,22.965278
2015-10-20 10:30:00,2035.0,2030.0,0.120705,0.128805,0.958333,22.958333
2015-10-20 10:40:00,2035.0,2030.0,0.121460,0.129155,0.951389,22.951389
2015-10-20 10:50:00,2040.0,2035.0,0.115975,0.126300,0.944444,22.944444
2015-10-20 11:00:00,2035.0,2035.0,0.124015,0.126630,0.937500,22.937500


#### Check for any Null values (NaNs)

In [41]:
options.isna().sum()

Strike1    0
Strike2    0
Option1    0
Option2    0
t1         0
t2         0
dtype: int64

#### Save final intraday SPX Options dataset to pickle 

In [41]:
options.to_pickle(intra_filepath + r'\Pickle\spx_intra_options.pkl')

### Generate 2015-2016 dataset request

In [70]:
spx_price = pd.read_csv(filepath + r'\SPX_index_close.2015-2016.csv', parse_dates=['DATE'], \
                         date_parser = lambda x: datetime.strptime(x,'%Y%m%d'))
spx_price.rename(columns={'DATE': 'date', 'spindx': 'spx_close'}, inplace=True)
spx_price.head()

,date,spx_close
0,2015-01-02,2058.20
1,2015-01-05,2020.58
2,2015-01-06,2002.61
3,2015-01-07,2025.90
4,2015-01-08,2062.14


In [84]:
spx_price[['Term_1', 'Term_2']] = spx_price['date'].apply(findTerms)

In [86]:
lo_moneyness = 0.85
hi_moneyness = 1.10
spx_price['lo_moneyness'] = lo_moneyness
spx_price['hi_moneyness'] = hi_moneyness
spx_price['lo_strike'] = round(spx_price['spx_close']*lo_moneyness)
spx_price['hi_strike'] = round(spx_price['spx_close']*hi_moneyness)
spx_price.tail()

,date,spx_close,Term_1,Term_2,lo_moneyness,hi_moneyness,lo_strike,hi_strike
499,2016-12-23,2263.79,2017-01-20,2017-02-17,0.85,1.1,1924.0,2490.0
500,2016-12-27,2268.88,2017-01-20,2017-02-17,0.85,1.1,1929.0,2496.0
501,2016-12-28,2249.92,2017-01-20,2017-02-17,0.85,1.1,1912.0,2475.0
502,2016-12-29,2249.26,2017-01-20,2017-02-17,0.85,1.1,1912.0,2474.0
503,2016-12-30,2238.83,2017-01-20,2017-02-17,0.85,1.1,1903.0,2463.0


In [88]:
spx_price.to_csv(filepath + '\Hamilton_dataset_request.2015-2016.csv', index=False)